In [1]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

# Tabular Modelling Deep Dive

Tabular modelling takes data in the form of a table (like a spreadsheet or CSV). The objective is to predict the value in one column based on the values in the other columns. In this chapter, we will not only look at deep learning but also more general machine learning techniques like random forests, as they can give better results depending on your problem.

We will look at how we should pre-process and clean the data, as well as how to interpret the result of our models after training.

## Categorical Embeddings

In tabular data, some columns/variables may be *continuous* whilst others may be *categorical*. Continuous variables are numerical data and can be directly fed to a model, since you can add and multiply them directly. Categorical variables contain a number of discrete levels, such as "movie ID", for which addition and multiplication don't have meaning.

The paper "Entity Embeddings for Categorical Variables" demonstrated one of the earliest known examples of a state-of-the-art deep learning tabular model. Entity embeddings are used to map similar values close to each other in the embedding space, revealing the intrinsic properties of categorical variables. They are used to transform categorical variables into inputs that are both continuous and meaningful. This is achieved by defining a distance measure for our categorical variables. It is especially useful for datasets with lots of high cardinality features and can be used for visualising categorical data and for data clustering.

The main insight from entity embeddings is that we can provide a model with fundamentally categorical data about discrete entities and that the model learns an embedding for these entities that defines a continuous notion of distance between them. Because the embedding distance was learned based on real patterns in the data, the distance tends to match up with our intuitions. In addition, it is valuable in its own right that embeddings are continuous, because models are better at understanding continuous variables. This is unsurprising considering that models are built of many continuous parameter weights and continuous activation values, which are updated via gradient descent.

Another benefit is that we can combine our continuous embedding values with truly continuous input data in a straightforward manner; we just concatenate the variables and feed the concatenation into our first linear layer. In other words, the raw categorical data is transformed by an embedding layer before it interacts with the raw continuous input data.

## Traditional Machine Learning vs. Deep Learning for Structured Data

Most modern machine learning can be distilled down to a few key techniques that are widely applicable. The vast majority of datasets can be best modelled with just two methods:
1. Ensembles of decision trees, mainly for structured data.
2. Multilayered neural networks learned with SGD, mainly for unstructured data.

Although deep learning is nearly always superior for unstructured data, these two approaches tend to give similar results for many kinds of structured data. But ensembles of decision trees tend to train faster, are often easier to interpret, do not require special GPU hardware for inference at scale, and often require less hyperparameter tuning.

Most importantly, the critical step of interpreting a model of tabular data is significantly easier for decision tree ensembles. There are tools and methods for answering the pertinent questions like:
- Which columns in the dataset were the most important for your predictions?
- How are they related to the dependent variable?
- How do they interact with each other?
- Which particular features were most important for some particular observation?
Therefore, ensembles of decision trees should be our first approach for analysing a new tabular dataset.

The exception to this guideline is when the dataset meets one of these conditions:
- There are some high-cardinality categorical variables that are very important (high number of discrete levels representing categories e.g. Postcode).
- There are some columns that contain data that would be best understood with a neural network, such as plain text data.

In practice, when we deal with datasets that meet these exceptional conditions, we always try both decision tree ensembles and deep learning to see which works best.

## Downloading the Data

The dataset we use in this chapter is from the Blue Book for Bulldozers Kaggle competition, which has the following description: "The goal of the contest is to predict the sale price of a particular piece of heavy equipment at auction based on its usage, equipment type, and configuration. The data is sourced from auction result postings and includes information on usage and equipment configurations."

This is a very common type of dataset and prediction problem, similar to what you may see in your project or workplace. The dataset is available for download on Kaggle, a website that hosts data science compitions.

The easiest way to download Kaggle datasets is to use the Kaggle API.

In [2]:
# Kaggle API Token
creds = '{"username": "hobz462", "key": "286dd5e15f0963c88007cd89dee68a56"}'

In [3]:
# Saving API Token locally
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
# Specifying a path to download the dataset to
comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)
path

Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers')

In [5]:
# Using the Kaggle API to download the dataset to that path and extracting it
from kaggle import api

if not path.exists():
    path.mkdir(parents=true)
    api.competition_download_cli(comp, path=path)
    shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

path.ls(file_type='text')

(#7) [Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/random_forest_benchmark_test.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/Valid.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/median_benchmark.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/Test.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/ValidSolution.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/Machine_Appendix.csv'),Path('/Users/286329i/.fastai/archive/bluebook-for-bulldozers/TrainAndValid.csv')]

## Looking at the Data
Kaggle provides information about some of the fields of our dataset. The [Data](https://www.kaggle.com/c/bluebook-for-bulldozers/data) explains that the key fields in *train.csv* are:

- `SalesID`:: The unique identifier of the sale.
- `MachineID`:: The unique identifier of a machine.  A machine can be sold multiple times.
- `saleprice`:: What the machine sold for at auction (only provided in *train.csv*).
- `saledate`:: The date of the sale.

In any sort of data science work, it's important to *look at your data directly* to make sure you understand the format, how it's stored, what types of values it holds, etc. Even if you've read a description of the data, the actual data may not be what you expect. We'll start by reading the training set into a Pandas DataFrame. Generally it's a good idea to specify `low_memory=False` unless Pandas actually runs out of memory and returns an error. The `low_memory` parameter, which is `True` by default, tells Pandas to only look at a few rows of data at a time to figure out what type of data is in each column. This means that Pandas can actually end up using different data type for different rows, which generally leads to data processing errors or model training problems later.

Let's load our data and have a look at the columns:

In [6]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)

In [7]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

That's a lot of columns for us to look at! Try looking through the dataset to get a sense of what kind of information is in each one. We'll shortly see how to "zero in" on the most interesting bits.

At this point, a good next step is to handle *ordinal columns*. This refers to columns containing strings or similar, but where those strings have a natural ordering. For instance, here are the levels of `ProductSize`:

In [8]:
df['ProductSize'].unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large', 'Compact'], dtype=object)

We can tell Pandas about a suitable ordering of these levels like so:

In [9]:
sizes = 'Large','Large / Medium','Medium','Small','Mini','Compact'

In [10]:
# Converting 'ProductSize' into an ordinal column
df['ProductSize'] = df['ProductSize'].astype('category')
df['ProductSize'].cat.set_categories(sizes, ordered=True)

0            NaN
1         Medium
2            NaN
3          Small
4            NaN
           ...  
412693      Mini
412694      Mini
412695      Mini
412696      Mini
412697      Mini
Name: ProductSize, Length: 412698, dtype: category
Categories (6, object): ['Large' < 'Large / Medium' < 'Medium' < 'Small' < 'Mini' < 'Compact']

The most important data column is the dependent variable—that is, the one we want to predict. Recall that a model's metric is a function that reflects how good the predictions are. It's important to note what metric is being used for a project. Generally, selecting the metric is an important part of the project setup. In many cases, choosing a good metric will require more than just selecting a variable that already exists. It is more like a design process. You should think carefully about which metric, or set of metrics, actually measures the notion of model quality that matters to you. If no variable represents that metric, you should see if you can build the metric from the variables that are available.

However, in this case Kaggle tells us what metric to use: root mean squared log error (RMSLE) between the actual and predicted auction prices. We need do only a small amount of processing to use this: we take the log of the prices, so that `rmse` of that value will give us what we ultimately need:

In [11]:
# Taking the log of SalePrice to use the metric RMSLE
dep_var = 'SalePrice'
df[dep_var] = np.log(df[dep_var])

## Handling Dates
The first piece of data representation we need to do is to enrich our representation of dates. You might want to treat a date as an ordinal value, because it is meaningful to say that one date is greater than the other. However, dates are a bit different from most ordinal values in that some dates are qualititatively different from others in a way that is often relevant to the systems we are modelling.

In order to help our algorithms handle dates intelligently, we'd like our model to know more than whether a date is more recent or less recent than another. We might want our model to make decisions based on that date's day of the week, on whether a day is a holiday, on what month it is in, and so forth. To do this, we replace every date column with a set of date metadata columns, such as holiday, day of week, and month. These columns provide categorical data that we suspect will be useful.

fastai comes with a function that will do this for us - we just have to pass a column name that contains dates:

In [12]:
# Creating new features from 'saledate'
df = add_datepart(df, 'saledate')

Let's do the same for the test set while we're here:

In [13]:
df_test = pd.read_csv(path/'Test.csv', low_memory=False)
df_test = add_datepart(df_test, 'saledate')

We can see that there are now lots of new columns in our DataFrame:

In [14]:
' '.join(o for o in df.columns if o.startswith('sale'))

'saleYear saleMonth saleWeek saleDay saleDayofweek saleDayofyear saleIs_month_end saleIs_month_start saleIs_quarter_end saleIs_quarter_start saleIs_year_end saleIs_year_start saleElapsed'

This is a good first step, but we will need to do a bit more cleaning. For this, we will use fastai objects called `TabularPandas` and `TabularProc`.

## Using TabluarPandas and TabularProc

A second piece of preparatory processing to be sure we can handle strings and missing data.

Out of the box, sklearn canot do either. Instead we will use fastai's class `TabularPandas`, which wraps a Pandas DataFrame and provides a few conveniences. To populate a `TabularPandas`, we will use two `TabularProcs`:
1. `Categorify`
2. `FillMissing`

A `TabularProc` is like a regular `Transform`, except that:
- It returns the exact same object that's passed to it, after modifying the object in place.
- It runs the transform once, when data is first passed in, rather than lazily as the data is accessed.

`Categorify` is a `TabularProc` that replaces a column with a numeric categorical column.

`FillMissing` is a `TabularProc` that replaces missing values with the median of the column, and creates a new Boolean column that is set to `True` for any row where the value was missing.

These two transforms are needed for nearly every tabular dataset you will use, so this is a good starting point for your data processing.

In [15]:
procs = [Categorify, FillMissing]

`TabularPandas` will also handle splitting the dataset into training and validation sets for us. However, we need to be very careful about our validation set. We want to design it so that it is like the *test* set Kaggle will use to judge the contest.

We don't get to see the test set. But we do want to define our validation data so that it has the same sort of relationship to the training data as the test set will have. In some cases, just randomly chosing a subset of your data oints will do that. This is not one of those cases, because it is a time series.

If you look at the data range represented in the test set, you will discover that it covers a six-month period from May 2012, which is later in time than any date in the training set. This is a good design, because the competition sponsor will want to ensure that the model is able to predict the future. But it means that if we are going to have a useful validation set, we also want the validation set to be later in time than the training set. The Kaggle training data ends in April 2012, so we will define a narrower training dataset which consists only of the Kaggle training data from before November 2011, and we'll define a validation set consisting of data from after November 2011.

To do this, we use `np.where`, a useful function that returns (as the first element of a tuple) the indices of all `True` values:

In [16]:
# Creating training and validation sets where the last 6 months are used for validation
cond = (df.saleYear<2011) | (df.saleMonth<10)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx), list(valid_idx))

`TabularPandas` needs to be told which columns are continuous and which are categorical. We can handle that automatically using the helper function `cont_cat_split`:


In [17]:
# Specifying our continuous and categorical columns
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [18]:
# Creating our TabularPandas object
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

A `TabularPandas` behaves a lot like a fastai `Datasets` object, including providing `train` and `valid` attributes:

In [19]:
len(to.train), len(to.valid)

(404710, 7988)

We can see that the data is still disabled as strings for categories (we only show a full columns here because the full table is too big):

In [20]:
to.show(5)

,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleElapsed,SalePrice
0,Low,521D,521,D,#na#,#na#,#na#,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,False,False,1139246,999089,3157,121,3.0,2004,68.0,2006,11,46,16,3,320,1.163635e+09,11.097410
1,Low,950FII,950,F,II,#na#,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,23.5,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,False,False,1139248,117657,77,121,3.0,1996,4640.0,2004,3,13,26,4,86,1.080259e+09,10.950807
2,High,226,226,#na#,#na#,#na#,#na#,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,OROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,False,False,False,False,False,False,1139249,434808,7009,121,3.0,2001,2838.0,2004,2,9,26,3,57,1.077754e+09,9.210340
3,High,PC120-6E,PC120,#na#,-6E,#na#,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metric Tons",Texas,TEX,Track Excavators,#na#,EROPS w AC,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,False,False,False,False,False,False,1139251,1026470,332,121,3.0,2001,3486.0,2011,5,20,19,3,139,1.305763e+09,10.558414
4,Medium,S175,S175,#na#,#na#,#na#,#na#,Skid Steer Loader - 1601.0 to 1751.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,EROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,False,False,False,False,False,False,1139253,1057373,17311,121,3.0,2007,722.0,2009,7,30,23,3,204,1.248307e+09,9.305651


However, the underlying items are all numeric:

In [21]:
to.items.head(5)

,SalesID,SalePrice,MachineID,ModelID,...,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,11.097410,999089,3157,...,1,1.163635e+09,1,1
1,1139248,10.950807,117657,77,...,1,1.080259e+09,1,1
2,1139249,9.210340,434808,7009,...,1,1.077754e+09,1,1
3,1139251,10.558414,1026470,332,...,1,1.305763e+09,1,1
4,1139253,9.305651,1057373,17311,...,1,1.248307e+09,1,1


The conversion of categorical columns to numbers is done by simply replacing each unique level with a number. The numbers associated with the levels are chosen consecutively as they are seen in a column, so there's no particular meaning to the numbers in categorical columns after conversion. The exception is if you first convert a column to a Pandas ordered category (as we did for `ProductSize` earlier), in which case the ordering you chose is used. We can see the mapping by looking at the `classes` attribute:

In [22]:
to.classes["ProductSize"]

['#na#', 'Compact', 'Large', 'Large / Medium', 'Medium', 'Mini', 'Small']

Since it takes a minute or so to process the data to get to this point, we should save it - that way in the future we can continue our work from here without rerunning the previous steps. 

fastai provides a `save` method that uses Python's *pickle* system to save nearly any Python object:

In [23]:
save_pickle(path/'to.pkl', to)

To read this back later, you would type:

In [24]:
to = load_pickle(path/'to.pkl')

In [25]:
to

        SalesID  SalePrice  MachineID  ModelID  ...  saleIs_year_start  \
0       1139246  11.097410     999089     3157  ...                  1   
1       1139248  10.950807     117657       77  ...                  1   
2       1139249   9.210340     434808     7009  ...                  1   
3       1139251  10.558414    1026470      332  ...                  1   
4       1139253   9.305651    1057373    17311  ...                  1   
...         ...        ...        ...      ...  ...                ...   
401120  6333336   9.259130    1840702    21439  ...                  1   
401121  6333337   9.305651    1830472    21439  ...                  1   
401122  6333338   9.350102    1887659    21439  ...                  1   
401123  6333341   9.104980    1903570    21435  ...                  1   
401124  6333342   8.955448    1926965    21435  ...                  1   

         saleElapsed  auctioneerID_na  MachineHoursCurrentMeter_na  
0       1.163635e+09                1     